In [3]:
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow as tf2
import os
from numpy.random import seed
from time import strftime
from PIL import Image
seed(888)
tf2.random.set_seed(404)
%load_ext tensorboard
!pip list | grep tensorflow

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
#Constants
X_TRAIN_PATH='./mnist/digit_xtrain.csv'
Y_TRAIN_PATH='./mnist/digit_ytrain.csv'
X_TEST_PATH='./mnist/digit_xtest.csv'
Y_TEST_PATH='./mnist/digit_ytest.csv'
TEST_IMG='./mnist/test_img.png'
LABEL_COUNT=10 #Number of digits i.e. 0-9
VALIDATION_SIZE=10000
IMG_WIDTH=28
IMG_HEIGHT=28
CHANNEL=1
TOTAL_INPUTS=784



In [5]:
#Retreive the data
xtrain=np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)
ytrain=np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int)
xtest=np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)
ytest=np.loadtxt(Y_TEST_PATH,delimiter=',',dtype=int)
print(xtrain.shape,ytrain.shape,xtest.shape,ytest.shape)

(60000, 784) (60000,) (10000, 784) (10000,)


In [6]:
#Data Preprocessing 
#converting the range between 0 and 1
xtrain,xtest=xtrain/255.0,xtest/255.0
#Creating a sparse matrix where the y value represent 1 on a single row of 10 labels
ytrain=np.eye(LABEL_COUNT)[ytrain]
ytest=np.eye(LABEL_COUNT)[ytest]
ytest.shape

(10000, 10)

In [7]:
#Creating Validation and Training Datasets
xval=xtrain[:VALIDATION_SIZE]
yval=ytrain[:VALIDATION_SIZE]
xtrain=xtrain[VALIDATION_SIZE:]
ytrain=ytrain[VALIDATION_SIZE:]
print(xtrain.shape,xval.shape)

(50000, 784) (10000, 784)


In [8]:
#Setting up Tensroflow graph
tf.disable_v2_behavior()
x=tf.placeholder(tf.float32,shape=[None,TOTAL_INPUTS],name='input-attr')
y=tf.placeholder(tf.float32,shape=[None,LABEL_COUNT])

Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
#Setting up the hyperparamters
epoch_count=20 #5 -> 50 -> 20
learning_rate=1e-3 # 1e-4 -> 1e-3
layer_1=512
layer_2=64


In [10]:
#Note : These layers can be moulded into a single function as well
with tf.name_scope('layer1'):
  #Setting up the layers  
  #LAYER #1
  #Setting the weights of each layer
  initial_w1=tf.truncated_normal(shape=[TOTAL_INPUTS,layer_1],stddev=0.1,seed=42)
  w1=tf.Variable(initial_value=initial_w1)
  #Setting the bias value
  initial_b1=tf.constant(value=0.0,shape=[layer_1])
  b1=tf.Variable(initial_value=initial_b1)
  #Can also use tf.summary.histogram to keep track of w and b for each layer
  #Constructing the first layer where it is the product of the placeholder and the input weights of the first layer
  layer_1_in=tf.matmul(x,w1)+b1
  #Treating the layer 1 result as activation function for the next layer
  layer_1_out=tf.nn.relu(layer_1_in)

In [11]:
with tf.name_scope('layer2'):
  #LAYER #2
  #Setting up the weights
  initial_w2=tf.truncated_normal(shape=[layer_1,layer_2],stddev=0.1,seed=42)
  w2=tf.Variable(initial_value=initial_w2)
  #Setting up the bias
  initial_b2=tf.constant(value=0.0,shape=[layer_2])
  b2=tf.Variable(initial_value=initial_b2)
  #Constructing the second layer
  layer_2_in=tf.matmul(layer_1_out,w2)+b2
  #Building the activation function
  layer_2_out=tf.nn.relu(layer_2_in)

In [12]:
with tf.name_scope('output'):
  #OUTPUT LAYER
  #Setting up weights
  #Final output will result into 10 labels
  initial_w3=tf.truncated_normal(shape=[layer_2,LABEL_COUNT])
  w3=tf.Variable(initial_value=initial_w3)
  #Setting up the bias
  initial_b3=tf.constant(value=0.0,shape=[LABEL_COUNT])
  b3=tf.Variable(initial_value=initial_b3)
  #Constructing the layer
  layer_3_in=tf.matmul(layer_2_out,w3)+b3
  #Building the activation function
  layer_3_out=tf.nn.softmax(layer_3_in)

In [13]:
#Loss Optimisation
#For performing batch operations,loss mean value is required 
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=layer_3_out))
#Setting up the optimizer
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
#Minimizing the loss function
train_step=optimizer.minimize(loss)

In [14]:
#Accuracy Metric
#Pick up the max probability index from last layer and compare with Y i.e. the original placeholder
with tf.name_scope('accuracy_calc'):
  model_output=tf.argmax(layer_3_out,axis=1,name='output-attr')
  correct_predictions=tf.equal(model_output,tf.argmax(y,axis=1))
  #casting to float to obtain mean value from all batches to be executed
  accuracy=tf.reduce_mean(tf.cast(correct_predictions,tf.float32))
with tf.name_scope('performance'):
  #Setting up the accuracy metric in summary
  tf.summary.scalar('accuracy',accuracy)
  tf.summary.scalar('loss',loss)

In [15]:
#Check input images in tensorboard
img=tf.reshape(x,[-1,28,28,1])
tf.summary.image('image_input',img,max_outputs=4)

<tf.Tensor 'image_input:0' shape=() dtype=string>

In [16]:
#Setting up folders for tensorboard event files 
folder_name=f'Model 1 at {strftime("%H %M")}'
dir_name=os.path.join('logs',folder_name)
try:
  os.makedirs(dir_name)
except OSError as err:
  print(err)

In [17]:
#Run Session
session=tf.Session()

#Creating Summaries
merged_summary=tf.summary.merge_all()
#Creates a subfilder
train_writer=tf.summary.FileWriter(dir_name+'/train')
#Creates a session graph
train_writer.add_graph(session.graph)
#For Validation session
validation_writer=tf.summary.FileWriter(dir_name+'/validation')

#Initializing all variables
init=tf.global_variables_initializer()
session.run(init)
#To check initialized values
b3.eval(session)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [18]:
#Creating Batches here we have 50k samples so we divide them into group of 1000s
batch_size=1000
num_examples=ytrain.shape[0]
iterations=int(num_examples/batch_size)
idx=0
iterations

50

In [19]:
def next_batch(batch_size,data,labels):
  global num_examples,idx
  start=idx
  idx+=batch_size
  end=idx
  if(idx>num_examples):
    start=0
    idx=batch_size
  return data[start:end],labels[start:end]


In [20]:
#Training Loop
#Number of passes for the whole dataset
for epoch in range(epoch_count):
  #Passing through each batch in dataset
  #####TRAINING DATASET #####
  for itr in range(iterations):
    batchx,batchy=next_batch(batch_size=batch_size,data=xtrain,labels=ytrain)
    feed_dictionary={x:batchx,y:batchy}
    #Running the session based on the data values
    session.run(train_step,feed_dict=feed_dictionary)
    #Summary for tensorboard and accuracy to directly print out 
  graph_summary_train,batch_accuracy=session.run(fetches=[merged_summary,accuracy],feed_dict=feed_dictionary)
  train_writer.add_summary(graph_summary_train,epoch)
  print(f'Epoch No. : {epoch} \t Accuracy is : {batch_accuracy}')
  ####VALIDATION DATASET ####
  graph_summary_val=session.run(fetches=merged_summary,feed_dict={x:xval,y:yval})
  validation_writer.add_summary(graph_summary_val,epoch)

print("Training Completed")

Epcoh No. : 0 	 Accuracy is : 0.7689999938011169
Epcoh No. : 1 	 Accuracy is : 0.8709999918937683
Epcoh No. : 2 	 Accuracy is : 0.8849999904632568
Epcoh No. : 3 	 Accuracy is : 0.972000002861023
Epcoh No. : 4 	 Accuracy is : 0.9810000061988831
Epcoh No. : 5 	 Accuracy is : 0.984000027179718
Epcoh No. : 6 	 Accuracy is : 0.9860000014305115
Epcoh No. : 7 	 Accuracy is : 0.984000027179718
Epcoh No. : 8 	 Accuracy is : 0.9850000143051147
Epcoh No. : 9 	 Accuracy is : 0.9909999966621399
Epcoh No. : 10 	 Accuracy is : 0.9900000095367432
Epcoh No. : 11 	 Accuracy is : 0.9879999756813049
Epcoh No. : 12 	 Accuracy is : 0.9919999837875366
Epcoh No. : 13 	 Accuracy is : 0.9919999837875366
Epcoh No. : 14 	 Accuracy is : 0.9909999966621399
Epcoh No. : 15 	 Accuracy is : 0.9919999837875366
Epcoh No. : 16 	 Accuracy is : 0.9900000095367432
Epcoh No. : 17 	 Accuracy is : 0.9919999837875366
Epcoh No. : 18 	 Accuracy is : 0.9919999837875366
Epcoh No. : 19 	 Accuracy is : 0.9919999837875366
Training Comp

In [21]:
#Saving attributes required to save the model
output_attr={'accuracy_calc/output-attr':model_output}
input_attr={'input-attr':x}
tf.saved_model.simple_save(session,'Saved MLP Model',input_attr,output_attr)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: Saved MLP Model\saved_model.pb


In [22]:
#Making Predictions
img=Image.open(TEST_IMG)
#convert this image to grayscale
img_gs=img.convert('L')
img_inv_gs=np.invert(img_gs) #Invert colors as input is black on white
final_img=img_inv_gs.ravel()
final_img.shape

(784,)

In [23]:
#sample prediction
prediction=session.run(fetches=tf.argmax(layer_3_out,axis=1),feed_dict={x:[final_img]})
print(prediction)

[2]


In [24]:
#Using the model on test dataset
test_acc=session.run(fetches=accuracy,feed_dict={x:xtest,y:ytest})
print(f'Test Accuracy : {test_acc:0.2%}')

Test Accuracy : 97.09%


In [25]:
#Resetting all calculations
train_writer.close()
validation_writer.close()
session.close()
tf.reset_default_graph()